# Homework II - Metodi quantitativi per la gestione del rischio
## Simona Maria Borrello s277789 
## Giulio Cerruto s277335 <a class="tocSkip">

## Introduzione
L'obiettivo dell'elaborato  è quello di valutare e confrontare alcune strategie di **immunizzazione di un portafoglio di bonds**, in modo tale che esso non risenta delle variazioni dei tassi di interesse. 

In particolare, si cercherà di fare luce sui seguenti temi: 
- immunizzazione al **primo ordine** vs immunizzazione al **secondo ordine**;
- impatto degli strumenti di hedging: **zero-coupon bond** vs **swap** ;
- effetto della **maturità** degli strumenti di hedging;
- influenza del **numero di strumenti** di hedging utilizzati.

Le diverse politiche saranno valutate in diversi contesti, in particolare nei casi in cui la struttura a termine dei tassi di interesse subisca: 
*  un **'piccolo'** shift **parallelo** ( *small pertubations* ) ; 
*  un **'grande'** shift **parallelo** ( *large pertubations* ); 
*  uno shift **non parallelo**, ottenuto in due modi diversi: 
    * l'aggiunta di un termine campionato da una distribuzione normale;
    * la variazione dei parametri che definiscono la struttura a termine dei tassi di interesse secondo il modello di Nelson–Siegel.

Prima di entrare nel vivo dell'analisi, implementiamo e illustriamo brevemente alcune funzioni che utilizzeremo nel seguito. 
Scegliamo, per semplicità di implementazione, di usare il continuous compounding e cedole semestrali nel caso di coupon bearing bonds e di swaps.

Infine, assumiamo che le **pertubazioni** della struttura dei tassi di interesse avvengano al tempo iniziale t=0 e siano **instantanee**.

In [1]:
import numpy as np
from tabulate import tabulate
import plotly.express as px
import plotly.graph_objects as go
from numpy import random

La funzione *price_zero* calcola il prezzo di un *zero-coupon bond*, dati i tassi di interesse r, il valore nominale F, i flussi di cassa, e la maturità T.

In [2]:
def price_zero(r_T,F,T): 
  return F* np.exp(-r_T* T)

La funzione *price_coupon_bond* calcola il prezzo di un *coupon bearing bond*, dati i tassi di interesse *r*, il tasso di cedola *c*, il valore nominale *F*, e la maturità *T*.

In [3]:
def price_coupon_bond(r, c, F, T):
  times = np.linspace(0.5,T,int(2*T)) # tempi dei flussi di cassa
  cf = (int(2*T)-1)*[c/2*F]
  cf.append((c/2+1)*F) # flussi di cassa
  df = np.exp(-times*r) # fattori di sconto
  P = np.dot(cf,df) # prezzo del bond
  return P

La funzione *compute_coupon_bond_duration* calcola la *duration* e la *$duration* di un *coupon bearing bond*, dati i tassi di interesse *r*, il tasso di cedola *c*, il valore nominale *F*, e la maturità *T*.

In [4]:
def compute_coupon_bond_duration(r, c, F, T):
  times = np.linspace(0.5,T,int(2*T)) # tempi dei flussi di cassa
  cf = (int(2*T)-1)*[c/2*F]
  cf.append((c/2+1)*F) # flussi di cassa
  df = np.exp(-times*r) # fattori di sconto
  P = np.dot(cf,df) # prezzo del bond
  duration = np.sum(times*df*cf)/P # duration del bond
  dollar_duration = duration*P # dollar duration
  return duration, dollar_duration

La funzione *compute_coupon_bond_convexity* calcola la *convexity* e la *$convexity* di un *coupon bearing bond*, dati i tassi di interesse *r*, il tasso di cedola *c*, il valore nominale *F*, e la maturità *T*.

In [5]:
def compute_coupon_bond_convexity(r, c, F, T):
  times = np.linspace(0.5,T,int(2*T)) # tempi dei flussi di cassa
  cf = (int(2*T)-1)*[c/2*F]
  cf.append((c/2+1)*F) # flussi di cassa
  df = np.exp(-times*r) # fattori di sconto
  P = np.dot(cf,df) # prezzo del bond
  convexity=  np.sum((times**2) *df*cf)/P
  dollar_convexity = convexity*P # dollar convexity
  return convexity, dollar_convexity


La funzione *find_swap_rate* trova il *par yield* di uno swap, cioè il rate per il cui il valore dello swap è zero all'inception. 

In [6]:
def find_swap_rate(r, T):  #trovare lo swap rate tale per cui il valore dello swap è zero all'inception
   delta= 0.5 #semplificazione, solo semiannuale
   times = np.linspace(0.5,T,int(2*T)) # tempi dei flussi di cassa
   z= [np.exp(-ra*t) for ra,t in zip(r,times)]
   sr = 1/delta * (1- np.exp(-r[-1] *T))/sum(z)
   return sr


La funzione *compute_swap_duration* calcola la *$duration* di uno swap ( per il floating), dati i tassi di interesse *r*, lo swap rate *sr*, il valore nominale *F*, la maturità *T*.

In [7]:
def compute_swap_duration(r, sr, F, T): #sr è lo swap rate 
  next_floating_payement= F* r[0]/2 #oppure r[0]
  dollar_duration_floating= next_floating_payement* 0.5 * np.exp(-r[0]*0.5)
  _, dollar_duration_fixed = compute_coupon_bond_duration(r, sr, F, T) 
  dollar_duration_swap= dollar_duration_floating - dollar_duration_fixed
  return dollar_duration_swap

La funzione *compute_Nelson_durations* calcola le 3 duration relative al modello parametrico di Nelson–Siegel, definite come *level duration*, *slope duration* e *curvature_duration*.

In [8]:
def compute_Nelson_durations(r, tau, times, cf):
  D=np.zeros(3)
  df = np.exp(-times*r) # fattori di sconto
  D[0] = -np.sum(times*df*cf)
  D[1]= -np.sum(times*df*cf* (1-np.exp(-times/tau))/(times/tau))
  D[2]= -np.sum(times*df*cf* ((1-np.exp(-times/tau))/(times/tau) - np.exp(-times/tau)))
  return D

La funzione *compute_phi* calcola il numero di zeri ( strumenti di hedging ) necessari per l'immunizzazione di un portafoglio di *N* zeri nel caso di un solo fattore di rischio *R* . 


In [9]:
#calcola il numero di zeri da usare come strumenti di hedging, dato un solo fattore di rischio R, il numero N di zeri del portafoglio, P i prezzi degli del portafoglio, D le duration degli zeri del portafoglio

def compute_phi(R,N,P,D, F):
  P_por= np.dot(N,P)
  W=[N[i]* P[i] / P_por for i in range(len(N))] #pesi del portafoglio
  D_portafoglio= np.dot(W,D) 
  T_H=R
  H= price_zero(r[T_H], F, T_H) #prezzo strumento di hedging
  D_H=  T_H
  H_shift= price_zero(r_shift[T_H], F, T_H)
  Phi= - D_portafoglio * P_por / (D_H * H)  
  return(Phi, H, H_shift) 

La funzione *compute_phi_nelson* calcola il numero di zeri ( strumenti di hedging ) necessari per l'immunizzazione di un portafoglio, data la variazione dei parametri del modello di Nelson–Siegel


In [10]:
def compute_phi_nelson(F, HT,r, r_shift, P_portafoglio, P_portafoglio_shift ): #HT= vettore contenente le maturità degli strumenti di hedging #F vettore contentente valori nominali degli strumenti

  level_duration_bond1, slope_duration_bond1, curvature_duration_bond1=compute_Nelson_durations([r[HT[0]]], tau, np.array([HT[0]]), F[0])
  level_duration_bond2, slope_duration_bond2, curvature_duration_bond2=compute_Nelson_durations([r[HT[1]]], tau, np.array([HT[1]]), F[1])
  level_duration_bond3, slope_duration_bond3, curvature_duration_bond3=compute_Nelson_durations([r[HT[2]]], tau, np.array([HT[2]]), F[2])
  a = np.array([[level_duration_bond1, level_duration_bond2,level_duration_bond3], [slope_duration_bond1, slope_duration_bond2, slope_duration_bond3],[curvature_duration_bond1, curvature_duration_bond2, curvature_duration_bond3]]) #dollar duration 1, dollar duration 2 , convexity dollar 1 convexity dollar 2
  b = np.array([ -level_duration_portafolio, -slope_duration_portafolio, -curvature_duration_portafolio]) #dollar duration portafoglio, convexity duration portafoglio
  [phi1, phi2, phi3] = np.linalg.solve(a, b)
  H=np.zeros(3)
  H_shift=np.zeros(3)
  H[0]= price_zero(r[HT[0]], F[0], HT[0]) #prezzo strumento di hedging
  H_shift[0]= price_zero(r_shift[HT[0]], F[0], HT[0]) #prezzo strumento di hedging

  H[1]= price_zero(r[HT[1]], F[1], HT[1]) #prezzo strumento di hedging
  H_shift[1]= price_zero(r_shift[HT[1]], F[1], HT[1])

  H[2]= price_zero(r[HT[2]], F[2], HT[2]) #prezzo strumento di hedging
  H_shift[2]= price_zero(r_shift[HT[2]], F[2], HT[2])

  P_H= P_portafoglio + phi1* H[0] + phi2* H[1] + phi3* H[2] 
  P_H_shift= P_portafoglio_shift + phi1 * (H_shift[0]) +  phi2 * (H_shift[1]) +  phi3 * (H_shift[2]) 
  differenza=  P_H_shift - P_H
  differenza_relativa= (differenza)/P_H*100
  return differenza, differenza_relativa

Adesso, costruiamo un portafoglio a cui applicare le tecniche di immunizzazione.
Scegliamo di includere i seguenti bonds:  
- **long** position su 2 zeri, con maturità 2 anni e valore nominale 10000$;

- **short** position su 4 zeri, con maturità 5 anni e valore nominale 5000$; 

- **long** position su 1 fixed coupon-bearing bond con maturità 2 anni,
valore nominale 10000$ e cedole semiannuali (c= 4%);

- **short** position su 4 zeri con maturità 1 anno e valore nominale 1000$.


In [11]:
#primo 
T1=2
F1=10000
#secondo
T2=5
F2=5000
#terzo
T3=2
F3= 10000
c=0.04
#quarto
T4=1
F4=1000
N=[2, -4 ,1, -4] #numero di bonds

La funzione *compute_value_portfolio* calcola il valore del nostro determinato portafoglio.

In [12]:
def compute_value_portfolio(r, N, F1,T1,F2,T2,F3,T3,F4,T4):
  P=np.zeros(4)
  P[0]= price_zero(r[T1], F1, T1)
  P[1]= price_zero(r[T2], F2, T2)
  P[2]=price_coupon_bond([r[0.5], r[1], r[1.5], r[2]], c,F3,T3)
  P[3]=  price_zero(r[T4], F4, T4) 
  return np.dot(N,P), P

#SHIFT PARALLELO (*small pertubations*)
## Struttura a termine dei tassi di interesse **costante**
Iniziamo dal caso più semplice: consideriamo una struttura a termine dei tassi **costante** (y=0.045) e assumiamo che ci sia uno **shift** di **-100 punti base.** Calcoliamo il valore,la duration e la convexity del portafoglio.



In [13]:
shift=-0.01
r={0.5 : 0.045, 1:0.045, 1.5: 0.045, 2:0.045, 2.5: 0.045, 3: 0.045, 3.5: 0.045, 4:0.045, 4.5: 0.045, 5:0.045, 5.5: 0.045, 6:0.045}
r_shift= {k:v+shift for k,v in r.items()} #shift parallelo di 100 punti base

fig = go.Figure()
fig.update_xaxes(title_text="T")
fig.update_yaxes(title_text="r(0,*)") 
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals =list(r.keys())))
fig.add_trace(go.Scatter(x=list(r.keys()), y=list(r.values()),
                    mode='lines+markers',
                    name='r')) 
fig.add_trace(go.Scatter(x=list(r_shift.keys()), y=list(r_shift.values()),
                    mode='lines+markers',
                    name='r-shift')) 
           
fig.show()
P_portafoglio, P =compute_value_portfolio(r, N, F1,T1,F2,T2,F3,T3,F4,T4) #valore portafoglio

D=np.zeros(4)
D[0]=T1 #duration di uno zereo
D[1]=T2
D[2], _ = compute_coupon_bond_duration([r[0.5], r[1], r[1.5], r[2]],c,F3,T3)
D[3]=T4

C=np.zeros(4)
C[0]=T1^2
C[1]=T2^2 #convexity di uno zero
C[2], _ = compute_coupon_bond_convexity([r[0.5], r[1], r[1.5], r[2]],c,F3,T3)
C[3]=T4^2
W=[N[i]* P[i] / P_portafoglio for i in range(4)] #pesi del portafoglio
D_portafoglio= np.dot(W,D) #duration del portafoglio
print(f'La duration del portafoglio vale {D_portafoglio:.2f}.') 
C_portafoglio= np.dot(W,C) #duration del portafoglio
print(f'La convexity del portafoglio vale  {C_portafoglio:.2f}.') 
print(f'Il valore del portafoglio, prima dello shift,  vale {P_portafoglio:.2f}$.') 

P_portafoglio_shift, P_shift =compute_value_portfolio(r_shift, N, F1,T1,F2,T2,F3,T3,F4,T4)
print(f'Il valore del portafoglio, dopo lo shift, vale {P_portafoglio_shift:.2f}$.')
print(f'La differenza tra i valori del portafoglio vale {P_portafoglio_shift - P_portafoglio:.2f}$, la differenza percentuale è di {(P_portafoglio_shift - P_portafoglio)/P_portafoglio:.2%}.')

La duration del portafoglio vale -3.33.
La convexity del portafoglio vale  -10.18.
Il valore del portafoglio, prima dello shift,  vale 8380.10$.
Il valore del portafoglio, dopo lo shift, vale 8086.15$.
La differenza tra i valori del portafoglio vale -293.95$, la differenza percentuale è di -3.51%.


Come notiamo, lo **shift** ha causato una **diminuizione** del valore del **portafoglio**. Per capirne il motivo, dobbiamo fare riferimento alla composizione del nostro portafoglio. In riferimento alla tabella sottostante, riportiamo le seguenti osservazioni: 
* chiaramente, la **diminuizione del rate** determina un **aumento di prezzo degli zeri**, rappresentando:

  *  una 'cattiva notizia' per gli zeri in posizione long;
  * una ' buona notizia' per gli zeri in posizione short; 
* la variazione è **più significativa** per **grandi maturità**;
*  le cedole del fixed coupon bond sono **reinvestite** ad un **tasso minore**;
* il prezzo del **fixed coupon bond** (c=0.04) è:
  * minore del suo valore nominale F=10000$, perchè **c< y**(=0.045), prima dello  shift;

  * maggiore del suo valore nominale F=10000$, perchè **c> y**(=0.035), dopo lo shift.

In [14]:
from tabulate import tabulate
percentuale= np.round( (P_shift- P)/ P,2 )
d = [['Zero long' ,P[0], P_shift[0], percentuale[0], T1],
    ['Zero short', P[1], P_shift[1], percentuale[1],T2],
    ['Fixed Coupon bond', P[2], P_shift[2],percentuale[2], T3],
    ['Zero short',P[3], P_shift[3],percentuale[3], T4]]

print(tabulate(d, headers=['Tipo', "Prezzo $", "Prezzo dopo lo shift $","%variazione" , "Maturità"]))

Tipo                 Prezzo $    Prezzo dopo lo shift $    %variazione    Maturità
-----------------  ----------  ------------------------  -------------  ----------
Zero long            9139.31                   9323.94            0.02           2
Zero short           3992.58                   4197.29            0.05           5
Fixed Coupon bond    9895.79                  10089.8             0.02           2
Zero short            955.997                   965.605           0.01           1




   L'obiettivo è quello di attuare una strategia di hedging in modo tale che la differenza tra i valori del portafoglio sia prossima a zero, che equivale a dire che il valore del nostro portafoglio non risente della variazione dei tassi di interesse. 
L'idea è quella che qualsiasi **perdita** (guadagno) del portafoglio sia **compensata** da un **guadagno** (perdita) dello strumento di hedging.

In questo caso il fattore di **rischio**, cioè lo shift parallelo della struttura dei tassi, è **uno solo** ed è quindi sufficiente introdurre un solo strumento di hedging. 

Se denotiamo con $H$ il valore dello strumento di hedging, con $D_H$ la sua duration, allora una *immunizzazione al primo ordine* del portafoglio si basa sull'inclusione di $\phi=- \frac{D_p \cdot P}{D_H \cdot H}$ strumenti di hedging (con opportune modifiche della formula per strumenti per cui non è definita la duration, come gli swap).

Per iniziare, scegliamo di utilizzare degli zero-coupond bond come strumenti di hedging. 


In [15]:
#primo caso 
F=1000
T_H=2
H= price_zero(r[T_H], F, T_H) #prezzo strumento di hedging
H_shift= price_zero(r_shift[T_H], F, T_H) #prezzo strumento di hedgind dopo lo shift

print(f'I prezzi dello strumento di hedging (uno zero con F= {F}$ e maturità {T_H}), prima e dopo lo shift, sono, rispettivamente, {H:.2f}$. e {H_shift:.2f}$.')
D_H=  T_H
print('La duration dello zero coincide con la sua maturità e vale ' + str( D_H) +'.')
Phi= - D_portafoglio * P_portafoglio / (D_H * H) 
print(f'Imponendo che la variazione del portafoglio, causata dallo shift, sia nulla, si trova che il numero di strumenti di hedging necessari è {Phi:.2f}.')
print(f'Inoltre, volendo ottenere un numero intero, il numero di zeri da includere nel portafoglio è {round(Phi, 0):.0f}.')
Phi= int(round(Phi, 0))
P_H= P_portafoglio + Phi* H

P_H_shift= P_portafoglio_shift + Phi * (H_shift) 
print(f'I valori del portafoglio con hedging P_H, prima e dopo lo shift, sono rispettivamente {P_H:.2f}$ e {P_H_shift:.2f}$.' )
differenza=  P_H_shift - P_H 
differenzarelativa= differenza/ P_H 
print(f'La loro differenza vale  {differenza:.2f}$, la differenza percentuale è {differenzarelativa*100:.5f}%.' )


I prezzi dello strumento di hedging (uno zero con F= 1000$ e maturità 2), prima e dopo lo shift, sono, rispettivamente, 913.93$. e 932.39$.
La duration dello zero coincide con la sua maturità e vale 2.
Imponendo che la variazione del portafoglio, causata dallo shift, sia nulla, si trova che il numero di strumenti di hedging necessari è 15.27.
Inoltre, volendo ottenere un numero intero, il numero di zeri da includere nel portafoglio è 15.
I valori del portafoglio con hedging P_H, prima e dopo lo shift, sono rispettivamente 22089.07$ e 22072.06$.
La loro differenza vale  -17.01$, la differenza percentuale è -0.07700%.


L'hedging sembra aver funzionato, dato che la perdita percentuale è di solo -0.077%.

## Struttura a termine dei tassi di interesse **crescente**
Quindi complichiamo lo scenario, cambiando la struttura a termine dei **tassi di interesse**. Assumiamo che essa sia non più costante, ma monotona **crescente**. 


In [16]:
r={0.5 : 0.02, 1:0.023, 1.5: 0.03, 2:0.035, 2.5: 0.036, 3: 0.04, 3.5: 0.043, 4:0.048, 4.5: 0.05, 5:0.051, 5.5: 0.052, 6:0.053}
fig = go.Figure()
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals =list(r.keys())))
fig.add_trace(go.Scatter(x=list(r.keys()), y=list(r.values()),
                    mode='lines+markers',
                    name='r')) 
fig.update_xaxes(title_text="T")
fig.update_yaxes(title_text="r(0,*)")            
fig.show()
P_portafoglio, P =compute_value_portfolio(r, N, F1,T1,F2,T2,F3,T3,F4,T4) #valore portafoglio

D=np.zeros(4)
D[0]=T1 #duration di uno zereo
D[1]=T2
D[2], _ = compute_coupon_bond_duration([r[0.5], r[1], r[1.5], r[2]],c,F3,T3)
D[3]=T4

C=np.zeros(4)
C[0]=T1^2
C[1]=T2^2 #convexity di uno zero
C[2], _ = compute_coupon_bond_convexity([r[0.5], r[1], r[1.5], r[2]],c,F3,T3)
C[3]=T4^2
print(f'Il valore del portafoglio è {P_portafoglio:.2f}$.') 
W=[N[i]* P[i] / P_portafoglio for i in range(4)] #pesi del portafoglio
D_portafoglio= np.dot(W,D) #duration del portafoglio
print(f'La duration del portafoglio vale {D_portafoglio:.2f}.') 
C_portafoglio= np.dot(W,C) #duration del portafoglio
print(f'La convexity del portafoglio vale  {C_portafoglio:.2f}.') 
D_dollar_portafoglio = D_portafoglio* P_portafoglio
C_dollar_portafoglio = C_portafoglio* P_portafoglio

Il valore del portafoglio è 9335.58$.
La duration del portafoglio vale -2.62.
La convexity del portafoglio vale  -8.73.


Assumiamo che si verifichi uno **shift** **parallelo** nella struttura dei tassi di interesse di **-100 punti base**. 

In [17]:
shift= -0.01
r_shift= {k:v+shift for k,v in r.items()} #shift parallelo di 100 punti base

fig = go.Figure()
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals =list(r.keys())))
fig.add_trace(go.Scatter(x=list(r.keys()), y=list(r.values()),
                    mode='lines+markers',
                    name='r')) 
fig.add_trace(go.Scatter(x=list(r_shift.keys()), y=list(r_shift.values()),
                    mode='lines+markers',
                    name='r-shift')) 
fig.update_xaxes(title_text="T")
fig.update_yaxes(title_text="r(0,*)")            
fig.show()

P_portafoglio_shift, P_shift =compute_value_portfolio(r_shift, N, F1,T1,F2,T2,F3,T3,F4,T4)
print(f'Il valore del portafoglio, prima lo shift, è {P_portafoglio:.2f}$.') 
print(f'Il valore del portafoglio, dopo lo shift, è {P_portafoglio_shift:.2f}$.')
print(f'La differenza tra i valori del portafoglio vale {P_portafoglio_shift - P_portafoglio:.2f}$, la differenza percentuale è di {(P_portafoglio_shift - P_portafoglio)/P_portafoglio:.2%}.')



Il valore del portafoglio, prima lo shift, è 9335.58$.
Il valore del portafoglio, dopo lo shift, è 9076.35$.
La differenza tra i valori del portafoglio vale -259.22$, la differenza percentuale è di -2.78%.


Anche in questo caso, lo shift ha causato una **diminuizione** del **valore** del portafoglio.   


L'**hedging** può essere attuato in **diversi** modi. Proviamo con i seguenti strumenti:
- uno zero con diverse maturità e  F=1000$ .

- uno swap con F= 1000$. 

##Strumento di hedging:  Zero coupond bond
Iniziamo con uno zero con maturità T=2.

In [18]:
#primo caso 
F=1000
T_H=2
H= price_zero(r[T_H], F, T_H) #prezzo strumento di hedging
H_shift= price_zero(r_shift[T_H], F, T_H)

print(f'I prezzi dello zero, prima e dopo lo shift, sono, rispettivamente, {H:.2f} $ e {H_shift:.2f}$.') 
D_H=  T_H
print('La duration dello zero coincide con la sua maturità e vale ' + str( D_H) +'.')
Phi= - D_portafoglio * P_portafoglio / (D_H * H) 
print(f'Imponendo che la variazione del portafoglio, causata dallo shift, sia nulla, si trova che il numero di strumenti di hedging necessari è {Phi:.2f}'+'.')
print(f'Inoltre, volendo ottenere un numero intero, il numero di zeri da includere nel portafoglio è {round(Phi, 0):.0f}.')
Phi= int(round(Phi, 0))
P_H= P_portafoglio + Phi* H
P_H_shift= P_portafoglio_shift + Phi * (H_shift) 
print(f'I valori del portafoglio con hedging, prima e dopo lo shift, sono rispettivamente {P_H:.2f}$ e {P_H_shift:.2f}$.' )
differenza=  P_H_shift - P_H 
differenzarelativa= differenza/ P_H 
primo=differenzarelativa *100
print(f'La loro differenza vale  {differenza:.2f}$, la differenza percentuale è {differenzarelativa*100:.5f}%.' )

I prezzi dello zero, prima e dopo lo shift, sono, rispettivamente, 932.39 $ e 951.23$.
La duration dello zero coincide con la sua maturità e vale 2.
Imponendo che la variazione del portafoglio, causata dallo shift, sia nulla, si trova che il numero di strumenti di hedging necessari è 13.14.
Inoltre, volendo ottenere un numero intero, il numero di zeri da includere nel portafoglio è 13.
I valori del portafoglio con hedging, prima e dopo lo shift, sono rispettivamente 21456.70$ e 21442.34$.
La loro differenza vale  -14.36$, la differenza percentuale è -0.06692%.


La **differenza** del portafoglio con hedging, prima  e dopo lo shift, è pressochè **nulla**. 
Proviamo adesso a variare la maturità dello zero.   


In [19]:

times= [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]
differenze_zero=np.zeros(len(times))
differenzerelativa=np.zeros(len(times))

for i,maturity in enumerate(times): 
  T_H= maturity
  H= price_zero(r[T_H], F, T_H) #prezzo strumento di hedging
  H_shift= price_zero(r_shift[T_H], F, T_H)

  D_H=  T_H
  Phi= - D_portafoglio * P_portafoglio / (D_H * H) 
  Phi= int(round(Phi, 0))
  P_H= P_portafoglio + Phi* H

  P_H_shift= P_portafoglio_shift + Phi * (H_shift) 
  differenze_zero[i]=  P_H_shift - P_H 
  differenzerelativa[i]= differenze_zero[i]/ P_H 
fig = go.Figure(data=[
    go.Bar(x=np.arange(1,13), y=differenzerelativa)
])


fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(1,13),
        ticktext = ['0.5', '1', '1.5', '2', '2.5', '3', '3.5', '4', '4.5', '5', '5.5', '6']
    )
)
fig.update_yaxes(title_text="Differenza % di valore del portafoglio")
fig.update_xaxes(title_text="Maturità dello strumento di hedging")  
fig.show()

L'hedging risulta soddisfacente in tutti i casi. 
L'andamento non è monotono, ma la tendenza generale è che la differenza% aumenta all'aumentare della maturità dello strumento di hedging.

##Strumento di hedging:  Swap
Adesso, proviamo a utilizzare come strumento di hedging uno swap.  
Dato che la **dollar duration** ( per fixed) , è tipicamente **negativa**,  esso rappresenta un ottimo strumento per ridurre la duration di un portafoglio. 

Il numero di strumenti di hedging necessari è $ \Phi= - \frac{ D_P^{\$} }{D_H^{\$}} $. 

Iniziamo con uno swap di maturità T=3. 

Per prima cosa, calcoliamo il rate per cui il valore iniziale dello swap è zero.  Per questo motivo, stipulare uno swap **non** necessita di **capitale aggiuntivo**.

In [20]:
# hedging con swap
T_H=3
sr= find_swap_rate([r[0.5], r[1], r[1.5], r[2], r[2.5], r[3]], T_H)

print(f'Lo swap rate vale {sr:.2f}.' )
F=1000
D_H=compute_swap_duration([r[0.5], r[1], r[1.5], r[2], r[2.5], r[3]], sr, F, T_H)
print(f'La dollar duration vale {D_H:.2f} $.' )

Phi= - D_portafoglio  * P_portafoglio / (D_H)

print(f'Imponendo che la variazione del portafoglio sia nulla, si trova che  il numero di swap necesseri è {Phi:.2f}. Chiaramente, volendo ottenere un numero intero, il numero di unità di strumenti da includere è {round(Phi, 0):.0f}.')
Phi= int(round(Phi, 0))

Lo swap rate vale 0.04.
La dollar duration vale -2850.06 $.
Imponendo che la variazione del portafoglio sia nulla, si trova che  il numero di swap necesseri è -8.60. Chiaramente, volendo ottenere un numero intero, il numero di unità di strumenti da includere è -9.


 Per verificare che l'hedging funzioni, calcoliamo la differenza nel valore del portafoglio e la differenza nel valore dello swap. 
 
 Esse, nel caso di un perfetto hedging,  dovrebbero essere uguali, ma di segno opposto.

In [21]:

diff= P_portafoglio_shift - P_portafoglio
print(f'La differenza, causata dallo shift nella struttura dei tassi, tra i valori del portafoglio vale {diff:.2f}$.')

diff2= - shift* D_H * Phi
print(f'La differenza tra i valori dello swap vale {diff2:.2f}$.')
print(f'L\'uso dello swap come strumento di hedging lascia scoperti per un valore di {abs(diff+diff2):.2f}$.')


La differenza, causata dallo shift nella struttura dei tassi, tra i valori del portafoglio vale -259.22$.
La differenza tra i valori dello swap vale 256.51$.
L'uso dello swap come strumento di hedging lascia scoperti per un valore di 2.72$.


Anche in questo caso, possiamo affermare che l'hedging ha funzionato.

### Confronto tra diversi strumenti di hedging
Confrontiamo le performance degli **zeri** e degli **swap**, al variare della maturità.

In [22]:
differenze_swap=np.zeros(len(times))
differenzerelativa=np.zeros(len(times))

for i,maturity in enumerate(times): 
  T_H=maturity
  r1=[r[x] for x in r.keys() if x<= maturity]
  sr= find_swap_rate(r1, T_H)
  F=1000
  D_H=compute_swap_duration(r1, sr, F, T_H)
  Phi= - D_portafoglio  * P_portafoglio / (D_H)
  Phi= int(round(Phi, 0))
  P_H_shift= P_portafoglio_shift + Phi * (H_shift) 
  diff= P_portafoglio_shift - P_portafoglio
  diff2= - shift* D_H * Phi
  differenze_swap[i]=  diff + diff2 

fig = go.Figure(data=[
    go.Bar(name='zero', x=np.arange(1,13), y=differenze_zero),
    go.Bar(name='swap', x=np.arange(1,13), y=differenze_swap)

])


fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(1,13),
        ticktext = ['0.5', '1', '1.5', '2', '2.5', '3', '3.5', '4', '4.5', '5', '5.5', '6']
    )
)
fig.update_yaxes(title_text="Differenza del valore del portafoglio")
fig.update_xaxes(title_text="Maturità dello strumento di hedging")  
fig.show()


A seconda della maturità, vediamo quale dei due strumenti di hedging performa meglio in termini assoluti. In generale, non è possibile scegliere a priori quale dei due è da preferire. 

Da notare che l'**hedging funziona** in bene in tutti i casi, ma ricordiamo che si tratta di una piccola pertubazione  del fattore di rischio. 

Vediamo, adesso, cosa succede in situazioni peggiori. 

#SHIFT PARALLELI (Large pertubations) 
##Immunizzazione al primo ordine: matching della duration

Immaginiamo che i tassi di interesse subiscano una **variazione maggiore**.

Quindi, ripetiamo la procedura vista sopra, con diversi valori di pertubazione.

In [23]:
shifts= [ +0.02,+0.025, +0.03,  +0.035, +0.04, +0.045]
base= [+200,+250,+300,+350, +400, +450]
differenze=np.zeros(len(base))
differenzerelativa=np.zeros(len(base))

fig = go.Figure()
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals =list(r.keys())))
fig.add_trace(go.Scatter(x=list(r.keys()), y=list(r.values()),
                    mode='lines+markers',
                    name='r')) 
for i,shift in enumerate(shifts): 
    
 
  r_shift= {k:v+shift for k,v in r.items()}
  fig.add_trace(go.Scatter(x=list(r_shift.keys()), y=list(r_shift.values()),
                      mode='lines+markers',
                      name='r-shift+ '+str(shift) )) 
  P_portafoglio_shift, P_shift =compute_value_portfolio(r_shift, N, F1,T1,F2,T2,F3,T3,F4,T4)

 #valore portafoglio con shift
  F=1000
  T_H= 4
  H= price_zero(r[T_H], F, T_H) #prezzo strumento di hedging
  D_H=  T_H
  Phi= - D_portafoglio * P_portafoglio / (D_H * H) 
  Phi= int(round(Phi,0))
  H_shift= price_zero(r_shift[T_H], F, T_H)
  P_H_shift= P_portafoglio_shift + Phi * (H_shift - H)
  P_H= P_portafoglio + Phi* H

  P_H_shift= P_portafoglio_shift +  Phi*H_shift
  differenze[i]= P_H_shift - P_H 
  differenzerelativa[i]= differenze[i]/ P_H *100


percentuale= np.round( differenzerelativa,2 )
d = [['+'+str(base[0]) , differenzerelativa[0]], ['+'+str(base[1]) , differenzerelativa[1]],   ['+'+str(base[2]) , differenzerelativa[2]],    ['+'+str(base[3]) , differenzerelativa[3]], ['+'+str(base[4]) , differenzerelativa[4]], ['+'+str(base[5]) , differenzerelativa[5]]]

print(tabulate(d, headers=['Variazione r','%variazione portafoglio']))
fig.update_xaxes(title_text="T")
fig.update_yaxes(title_text="r(0,*)")  
fig.show()

fig = go.Figure()
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals =list(r.keys())))
fig.add_trace(go.Bar(x=[0, 1, 2, 3, 4, 5, 6], y=differenze))


fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 1, 2, 3, 4, 5, ],
        ticktext = [  '+200','+250','+300','+350', '+400', '+450']
    )
)
fig.update_yaxes(title_text="Differenza di valore del portafoglio")
fig.update_xaxes(title_text="Variazione di r in punti base ")  
fig.show()


  Variazione r    %variazione portafoglio
--------------  -------------------------
          +200                 -0.0496039
          +250                 -0.130821
          +300                 -0.237567
          +350                 -0.368893
          +400                 -0.523879
          +450                 -0.701628


Come ci si aspetta, più **significativa** è la **variazione** dei tassi di interesse, **peggiore** è la performance della politica di **hedging**. 


##Immunizzazione al secondo ordine: matching di convexity e duration
Per migliorare la qualità del nostro approccio, proviamo a raffinare l'immunizzazione utilizzando una **approssimazione al secondo ordine**, in modo tale da considerare anche la sensitività della duration rispetto alle variazioni dei tassi di interesse.

L'obiettivo è quello  di ottenere un portafoglio che sia *duration-neutral*  e *convexity-neutral*.

Utilizziamo due strumenti di hedging e risolviamo il seguente sistema lineare: 
$$ 
\begin{cases}
 \phi_1 \$D_1 + \phi_2 \$D_2 &= - \$D_P \\
  \phi_1 \$C_1 + \phi_2 \$C_2 &= - \$C_P 
\end{cases}
 $$


In [24]:
differenza_con=np.zeros(len(base))
differenzerelativa_con=np.zeros(len(base))
for i,shift in enumerate(shifts): 
    
 
  r_shift= {k:v+shift for k,v in r.items()}
  fig.add_trace(go.Scatter(x=list(r_shift.keys()), y=list(r_shift.values()),
                      mode='lines+markers',
                      name='r-shift+ '+str(shift) )) 
  P_portafoglio_shift, P_shift =compute_value_portfolio(r_shift, N, F1,T1,F2,T2,F3,T3,F4,T4)

 #valore portafoglio con shift
  F=1000
  T_H1= 4.5
  DH1=T_H1
  CH1= T_H1**2
  H1= price_zero(r[T_H1], F, T_H1) #prezzo strumento di hedging

  T_H2= 0.5
  DH2=T_H2
  CH2= T_H2**2

  H2= price_zero(r[T_H2], F, T_H2) #prezzo strumento di hedging
  a = np.array([[DH1*H1, DH2*H2], [CH1* H1, CH2*H2]]) #dollar duration 1, dollar duration 2 , convexity dollar 1 convexity dollar 2
  b = np.array([ -D_dollar_portafoglio, -C_dollar_portafoglio]) #dollar duration portafoglio, convexity duration portafoglio
  [phi1, phi2] = np.linalg.solve(a, b)
  phi1= int(round(phi1,0)) # numero di strumenti è intero
  phi2= int(round(phi2,0))
    
  H_shift1= price_zero(r_shift[T_H1], F, T_H1)
  H_shift2= price_zero(r_shift[T_H2], F, T_H2)
  P_H= P_portafoglio + phi1* H1+ phi2*H2

  P_H_shift= P_portafoglio_shift +  phi1* H_shift1+ phi2*H_shift2
  differenza_con[i]= P_H_shift -  P_H
  differenzerelativa_con[i]= differenze[i]/ P_H *100

#plot

fig = go.Figure(data=[
    go.Bar(name='duration immunization', x=np.arange(1,7),  y=differenzerelativa),
    go.Bar(name='convexity immunization', x=np.arange(1,7), y=differenzerelativa_con)])

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(1,7),
        ticktext = [  '+200','+250','+300','+350', '+400', '+450']
    )
)
fig.update_yaxes(title_text="Variazione % valore del portafoglio")
fig.update_xaxes(title_text="Variazione di r in punti base ")  
fig.show()




Come ci si può aspettare, l'hedging basato anche sulla **convexity** risulta migliore. Infatti l'approsimazione della relazione tra prezzi e tassi di interesse risulta **più** **precisa**.

# SHIFT NON PARALLELO
Adesso, complichiamo il quadro e ipotizziamo che lo shift della struttura **non** sia **parallelo**. \
 Gestire delle variazioni non parallele è più complicato, dato che la **dimensionalità** del problema **cresce**. 
In particolare, come vedremo, l'utilizzo di un solo strumento di hedging per più di un fattore di rischio condurrà a scenari di under-hedging. \
In generale, infatti nel caso di molteplici sorgenti di incertezza, risultano necessari diversi strumenti di hedging. 
## Shift non parallelo:  aggiunta di un termine random
Possiamo ottenere uno shift non parallelo della struttura a termine dei tassi aggiungendo ad ogni maturità un campione di una **variabile** aleatoria **normale**.
Nel nostro caso, si è campionato da una distribuzione normale di media $\mu=-0.01$ e deviazione standard $ \sigma= 0.01$ . 

Scegliamo di utilizzare una distrbuzione normale di media -0.01, in modo tale che sia il più lecito possibile confrontare i risultati che otteniamo con il caso iniziale di shift parallelo di -100 punti base.

Inoltre, per ottenere una struttura a termine crescente, e quindi più verosimile, si campiona fino a quando non si ottiene un tasso che sia maggiore di quello precedente.

In [25]:
random.seed(1000)
shift=np.linspace(0,0.02,10)
i=0
r_shift={}
for k,v, in r.items():
 # r_shift[k]=v+ shift[i]
    if k==0.5 : 
      r_shift[k]= r[k]+ np.random.normal (-0.01, 0.01)
    else: 
      r_shift[k]= r[k]+ np.random.normal  (-0.01, 0.01)

      while r_shift[k]<r_shift[k-0.5]:
          r_shift[k]= r[k]+ np.random.normal  (-0.01, 0.01)


fig = go.Figure()
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals =list(r.keys())))
fig.add_trace(go.Scatter(x=list(r.keys()), y=list(r.values()),
                    mode='lines+markers',
                    name='r')) 
fig.add_trace(go.Scatter(x=list(r_shift.keys()), y=list(r_shift.values()),
                    mode='lines+markers',
                    name='r-shift')) 
fig.update_xaxes(title_text="T")
fig.update_yaxes(title_text="r(0,*)")            
fig.show()

### 1  STRUMENTO DI HEDGING
Per prima cosa, proviamo ad utilizzare un solo strumento di hedging: uno zero di maturità T=3.  

In [26]:

P_portafoglio_shift, P_shift =compute_value_portfolio(r_shift, N, F1,T1,F2,T2,F3,T3,F4,T4)
#valore portafoglio con shift
F=1000
T_H=3
H= price_zero(r[T_H], F, T_H) #prezzo strumento di hedging
D_H=  T_H
Phi= - D_portafoglio * P_portafoglio / (D_H * H) 
Phi= int(round(Phi, 0))
H_shift= price_zero(r_shift[T_H], F, T_H)
P_H_shift= P_portafoglio_shift + Phi * (H_shift)
P_H= P_portafoglio + Phi*  H
differenza=   P_H- P_H_shift 
differenzarelativa= differenza/ P_H
secondo=differenzarelativa*100

print(f'La differenza tra i valori  del portafoglio con hedging vale  {differenza:.2f}$, la differenza percentuale è {differenzarelativa:.5%}.' )


La differenza tra i valori  del portafoglio con hedging vale  -718.66$, la differenza percentuale è -4.14980%.


In questo caso, la **differenza** percentuale è **significativa**.

 L'utilizzo di un solo strumento di hedging sembra **non** essere **sufficiente** per una ottima copertura. 

###5 STRUMENTI DI HEDGING
Adesso, sperimentiamo una strategia di hedging più accurata e introduciamo **tanti strumenti** di hedging **quanti** sono **i fattori di rischio.** Nel nostro caso i fattori di rischio R sono 5, perchè il valore del portafoglio dipende da 5 diversi tassi di interesse: r(0, 0.5) , r(0, 1), r(0, 1.5), r(0, 2) , r(0.5). 
L'approssimazione della variazione del valore del portafoglio coperto è
$$ \delta P^{H}= \sum_{i=1}^5  \biggl( \frac{\partial
 P }{\partial R_i }+ \sum_{j=1}^5 \phi _j  \frac{\partial H_j}{\partial R_i} \biggr) \delta R_i  $$
 Per ottenere le derivate parziali rispetto ai fattori di rischio, sfruttiamo come prima la conoscenza di una esplicita formula di pricing. 
 
 Per semplicità, come strumenti di hedging, scegliamo di usare degli zeri con 5 diverse maturità corrispondenti a quelle dei fattori di rischio. Di conseguenza, si ottiene una grande **semplificazione** perchè il **sistema** di equazioni sarà **disaccopiato**, in quanto ogni strumento di hedging dipenderà da un unico fattore di rischio.

 Per calcolare il numero di strumenti necessari, scomponiamo il portafoglio e calcoliamo i flussi di cassa associati ad ogni maturità. 

Il primo fattore di rischio R1= r(0, 0.5) coinvolge solo il primo stacco di cedola del fixed coupon bond del nostro portafoglio. 

Esso può essere trattato come uno zero che ha come valore nominale c*F. 

In [27]:
F=100
R1=0.5
D1=[R1]
P1=[price_zero(r[R1], c*F3, R1)]
phi1, H1,  H1_shift = (compute_phi(R1, [N[2]], P1, D1, F))
print(f'Il numero di strumenti di hedging (zeri con maturità T= {R1}) da includere nel portafoglio è {int(phi1)}')

Il numero di strumenti di hedging (zeri con maturità T= 0.5) da includere nel portafoglio è -4


Il secondo fattore di rischio R2=r(0, 1) influenza la seconda maturità del fixed coupon bond del nostro portafoglio e i 5 zeri del portafoglio che hanno maturità 1. Quindi il portafoglio complessivo può essere considerato come costituito da 6 zeri.  

In [28]:
R2=1
D2=[R2, R2]
P2=[price_zero(r[R2], c*F3, R2), price_zero(r[R2], F4, R2)]
phi2, H2,  H2_shift = (compute_phi(R2, [N[2], N[3]], P2, D2, F))
print(f'Il numero di strumenti di hedging (zeri con maturità T= {R2}) da includere nel portafoglio è {int(phi2)}.')

Il numero di strumenti di hedging (zeri con maturità T= 1) da includere nel portafoglio è 36.


Il terzo fattore di rischio R2=r(0, 1.5) influenza solo la terza maturità del fixed coupon bond del nostro portafoglio.  Esso può essere trattato come uno zero che ha come valore nominale c*F. 

In [29]:
R3= 1.5
P3=[price_zero(r[R3], c*F3, R3)]
D3=[R3]
phi3, H3,  H3_shift = (compute_phi(R3, [N[2]], P3, D3,F))
print(f'Il numero di strumenti di hedging (zeri con maturità T= {R3}) da includere nel portafoglio è {int(phi3)}')

Il numero di strumenti di hedging (zeri con maturità T= 1.5) da includere nel portafoglio è -4


Il quarto fattore di rischio R2=r(0, 2) influenza la quarta e ultima maturità del fixed coupon bond ( e viene trattato come uno zero dal valore nominale (c+1)*F) e lo zero del portafoglio che ha maturità 2. Quindi il portafoglio complessivo può essere considerato come costituito da 2 zeri. 

In [30]:
R4= 2
N4=[1, 1] #numero di bonds
P4=[price_zero(r[R4], (c+1)*F3, R4), price_zero(r[R4], F1, R4)]
D4=[R4, R4]
phi4, H4,  H4_shift = (compute_phi(R4, [N[2], N[0]], P4, D4,F))
print(f'Il numero di strumenti di hedging (zeri con maturità T= {R4}) da includere nel portafoglio è {int(phi4)}.')

Il numero di strumenti di hedging (zeri con maturità T= 2) da includere nel portafoglio è -304.


L'ulitmo fattore di rischio R5=r(0, 5) influenza solo i 2 zeri del portafoglio che hanno maturità 5. 

In [31]:
R5= 5
P5=[price_zero(r[R5], F2, R5)]
D5=[R5]
phi5 ,H5,  H5_shift = (compute_phi(R5, [N[1]], P5, D5,F))
print(f'Il numero di strumenti di hedging (zeri con maturità T= {R5}) da includere nel portafoglio è {int(phi5)}.')

Il numero di strumenti di hedging (zeri con maturità T= 5) da includere nel portafoglio è 200.


In [32]:
phi1= int(round(phi1, 0))
phi2= int(round(phi2, 0))
phi3= int(round(phi3, 0))
phi4= int(round(phi4, 0))
phi5= int(round(phi5, 0))


P_H_shift= P_portafoglio_shift + phi1 * (H1_shift) +phi3 * (H3_shift) +phi4 * (H4_shift) + phi5 * (H5_shift) + phi2*(H2_shift)
P_H= P_portafoglio + phi1 * H1 + phi2 * H2 + phi3 * H3 + phi4 * H4 + phi5 * H5
differenza=  P_H_shift - P_H

differenzarelativa= differenza/ P_H 
terzo= differenzarelativa *100
print(f'La differenza tra i valori del portafoglio con hedging vale {-differenza:.2f}$, la differenza relativa rispetto al valore iniziale è {differenzarelativa:.5f}.' )

La differenza tra i valori del portafoglio con hedging vale 7.42$, la differenza relativa rispetto al valore iniziale è 0.00962.


Come vediamo l'hedging con più di uno strumento di hedging performa meglio, dato che esso tiene conto di tutti i fattori di rischio.

In [33]:
fig = go.Figure()

fig.add_trace(go.Bar(x=[0, 1, 2], y=[primo, secondo, terzo]))


fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 1, 2],
        ticktext = ['Shift parallelo (- 100 punti base) ','Shift non parallelo- 1 strumento di hedging','Shift non parallelo- 5 strumenti di hedging']
    )
)
fig.update_yaxes(title_text="Differenza% valore del portafoglio")
#fig.update_xaxes(title_text="Maturità degli strumenti di hedging ")  
fig.show()

Dal confronto, emerge chiaramente che la copertura del portafoglio con un unico strumento di hedging determina uno scenario di **underhedging** nel caso di **molteplici** fattori di **rischio**, mentre risulta **adeguata** nel caso di un **unico** fattore di **rischio**.

## Shift non parallelo: variazione dei parametri del modello di Nelson e Siegel 
Utilizziamo ora un approccio parametrico per la struttura a termine dei tassi. In particolare, facciamo riferimento al modello di Nelson e Siegel i cui parametri $ \beta_0, \beta_1, \beta_2 $ costituiscono rispettivamente l'intercetta, il coefficiente angolare e la curvatura della curva dei tassi, mentre il parametro $ \tau $ è un parametro di scala.

Per il nostro modello, scegliamo i seguenti parametri:

  $ \beta_0= 0.08,$     $\beta_1= -0.03,$   $ \beta_2=-0.01,$  $\tau= 3 $. 

  Dati questi parametri, mostriamo la risultante struttura a termine dei tassi di interesse nel grafico sottostante.  

In [34]:
times=[0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5,5, 5.5, 6]

b0=0.08
b1=-0.03
b2=-0.01
tau=3
r=dict()
for t in times:
  r[t]= b0 + b1*((1- np.exp(-t/tau ))/(t/tau)) + b2*((1-np.exp(- t/tau))/(t/tau) - np.exp(- t/tau) )

fig = go.Figure()
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals =list(r.keys())))
fig.add_trace(go.Scatter(x=list(r.keys()), y=list(r.values()),
                    mode='lines+markers',
                    name='r')) 
fig.update_xaxes(title_text="T")
fig.update_yaxes(title_text="r(0,*)")            
fig.show()


Per interpretare in modo semplice le sensitività $\beta$, possiamo considerare un fixed coupon rate (c=5%, frequenza annuale) , maturità 2 anni e valore nominale F=100. Quindi calcoliamo le diverse durations del modello di Nelson. 

In [35]:
cashflows= [100*0.05, 100*(1.05)]
level_duration, slope_duration, curvature_duration=compute_Nelson_durations([ r[1], r[2]], tau, np.array([1,2]), cashflows)
print(f' D_0= {level_duration:.2f}, D_1= {slope_duration:.2f}, D_2= {curvature_duration:.2f}')

 D_0= -192.51, D_1= -141.08, D_2= -41.28


Ad esempio, $D_0$ indica che, se $\beta_0$ aumenta dell' 1%, allora il prezzo del bond diminuirà perchè $ D_0$ è negativo e la variazione sarà di  1.9251 \$ , cioè di $ D_0$ * 1%. 
Analoghi ragionamenti si applicano agli altri parametri.

Quindi, introduciamo alcune variazioni ai parametri del modello. 
In particolare: 

  $ \beta_{0,shift}= \beta_0 -0.04,$     
  $ \beta_{1,shift}= \beta_1+  0.01,$     
  $ \beta_{2,shift}= \beta_2 +0.15,$    

In [36]:
b0_shift=b0- 0.04
b1_shift=b1+ 0.01
b2_shift=b2+ 0.15
r_shift=dict()
for t in times:
  r_shift[t]= b0_shift + b1_shift*((1- np.exp(-t/tau ))/(t/tau)) + b2_shift*((1-np.exp(- t/tau))/(t/tau) - np.exp(- t/tau) )

fig = go.Figure()
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals =list(r.keys())))
fig.add_trace(go.Scatter(x=list(r.keys()), y=list(r.values()),
                    mode='lines+markers',
                    name='r'))
fig.add_trace(go.Scatter(x=list(r_shift.keys()), y=list(r_shift.values()),
                    mode='lines+markers',
                    name='r-shift'))  
fig.update_xaxes(title_text="T")
fig.update_yaxes(title_text="r(0,*)")            
fig.show()

Il numero di parametri del modello equivale al  numero di fattori di rischio, che a sua volta corrisponde al numero di strumenti di hedging necessari. 

Per trovare il numero di strumenti $ \phi_1, \phi_2 \phi_3$ risolviamo il seguente sistema di equazioni:  
$$
\begin{cases}
\phi_1 \frac{ \partial{H_1}}{\partial{\beta_0}} + \phi_2 \frac{ \partial{H_2}}{\partial{\beta_0}}+ \phi_3 \frac{ \partial{H_3}}{\partial{\beta_0}}&= - D_0 \\
\phi_1 \frac{ \partial{H_1}}{\partial{\beta_1}} + \phi_2 \frac{ \partial{H_2}}{\partial{\beta_1}}+ \phi_3 \frac{ \partial{H_3}}{\partial{\beta_1}}&= - D_1 \\
\phi_1 \frac{ \partial{H_1}}{\partial{\beta_2}} + \phi_2 \frac{ \partial{H_2}}{\partial{\beta_2}}+ \phi_3 \frac{ \partial{H_3}}{\partial{\beta_2}}&= - D_2
\end{cases}
 $$

 
Per iniziare, utilizziamo come strumenti di hedging 3 zeri di maturità 1,3, 6 e F=1000$.


In [37]:
P_portafoglio, P =compute_value_portfolio(r, N, F1,T1,F2,T2,F3,T3,F4,T4) #valore portafoglio
cashflows= [N[2]*c*F3, N[2]*c*F3+N[3]*F4, N[2]*c*F3, N[2]*(c+1)*F3+ N[0]*F1, N[1]*F2 ]
level_duration_portafolio, slope_duration_portafolio, curvature_duration_portafolio=compute_Nelson_durations([ r[0.5], r[1], r[1.5], r[2], r[5]], tau, np.array([0.5, 1,1.5,2, 5]), cashflows)
P_portafoglio_shift, _ =compute_value_portfolio(r_shift, N, F1,T1,F2,T2,F3,T3,F4,T4) #valore portafoglio

F=[1000, 1000, 1000]
HT=[0.5, 4, 5]
differenza, differenzarelativa = compute_phi_nelson(F,HT,r, r_shift, P_portafoglio, P_portafoglio_shift)
print(f'La differenza tra i valori  del portafoglio con hedging vale {differenza:.2f}$, la differenza% è di {differenzarelativa:.5}%.' )

La differenza tra i valori  del portafoglio con hedging vale -9.02$, la differenza% è di 0.055008%.


La copertura del portafoglio ottenuta risulta soddisfacente.
Quindi vediamo come le maturità degli strumenti di hedging utilizzati influenzano la performance dell'hedging. 

Proviamo alcune triplette di maturità  e confrontiamo i risultati.

In [38]:
F=[1000, 1000, 1000]
maturità=[[0.5,1, 1.5],[1,1.5,2],[1,2,3],[0.5,2 ,6],[1, 2,4] ,[0.5, 2, 3], [3,3.5,4],[0.5, 4, 5],[1,5,6] , [2, 4, 5], [3, 4, 5],[4,5,6],[4.5, 4, 6], [5, 5.5, 6]]
differenza= np.zeros(len(maturità))
differenzarelativa= np.zeros(len(maturità))

for i, HT in enumerate(maturità):
  differenza[i], differenzarelativa[i] = compute_phi_nelson(F,HT,r, r_shift, P_portafoglio, P_portafoglio_shift)
fig = go.Figure()
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals =list(r.keys())))
fig.add_trace(go.Bar(x=np.arange(1,15), y=differenzarelativa))


fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(1,15),
        ticktext = ['[0.5, 1, 1.5]','[1, 1.5, 2]','[1, 2, 3]','[0.5, 2 ,6]','[1, 2, 4]' ,'[0.5, 2, 3]', '[3, 3.5, 4]','[0.5, 4, 5]','[1,5,6]' , '[2, 4, 5]', '[3, 4, 5]','[4, 5, 6]','[4.5, 4, 6]', '[5, 5.5, 6]']
    )
)
fig.update_yaxes(title_text="Differenza valore del portafoglio")
fig.update_xaxes(title_text="Maturità degli strumenti di hedging ")  
fig.show()

In generale, l'hedging risulta efficiente, sebbene, come si può evincere, alcune combinazioni siano più perfomanti di altre.
Ora, analizziamo diverse variazioni della struttura a termine dei tassi di interesse.

In [39]:
numeri_scenari=5

differenza= np.zeros((len(maturità), numeri_scenari))
differenzarelativa= np.zeros((len(maturità), numeri_scenari))

fig = go.Figure()

b0_shift_vec=[-0.03, 0, 0.01, 0.01, -0.001 ]
b1_shift_vec=[0.03, -0.02, 0, 0.01, 0.04]
b2_shift_vec=[0.01, -0.01, 0.08, 0.05, 0.01]

for j in range(numeri_scenari):
  r_shift=dict()
  b0_shift= b0+b0_shift_vec[j]
  b1_shift= b1+b1_shift_vec[j]
  b2_shift=b2+ b2_shift_vec[j]

  for t in times:
    r_shift[t]= b0_shift + b1_shift*((1- np.exp(-t/tau ))/(t/tau)) + b2_shift*((1-np.exp(- t/tau))/(t/tau) - np.exp(- t/tau) )


  fig.add_trace(go.Scatter(x=list(r_shift.keys()), y=list(r_shift.values()),
                      mode='lines+markers',
                      name='r-shift'+ str(j+1)))  
  
  P_portafoglio_shift, _ =compute_value_portfolio(r_shift, N, F1,T1,F2,T2,F3,T3,F4,T4) 
  
  for i, HT in enumerate(maturità):
    differenza[i,j], differenzarelativa[i,j] = compute_phi_nelson(F,HT,r, r_shift, P_portafoglio, P_portafoglio_shift)            
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals =list(r.keys())))
fig.add_trace(go.Scatter(x=list(r.keys()), y=list(r.values()),
                    mode='lines+markers',
                    name='r'))
fig.update_xaxes(title_text="T")
fig.update_yaxes(title_text="r(0,*)") 
fig.show()


fig = go.Figure()
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals =list(r.keys())))
fig = go.Figure(data=[
    go.Bar(name='shift-1', x=np.arange(1,15), y=differenzarelativa[:,0]), go.Bar(name='shift-2', x=np.arange(1,15), y=differenzarelativa[:,1]),go.Bar(name='shift-3', x=np.arange(1,15), y=differenzarelativa[:,2]),
    go.Bar(name='shift-4', x=np.arange(1,15), y=differenzarelativa[:,3]), go.Bar(name='shift-5',  x=np.arange(1,15), y=differenzarelativa[:,4])
])
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(1,15),
        ticktext = ['[0.5, 1, 1.5]','[1, 1.5, 2]','[1, 2, 3]','[0.5, 2 ,6]','[1, 2, 4]' ,'[0.5, 2, 3]', '[3, 3.5, 4]','[0.5, 4, 5]','[1,5,6]' , '[2, 4, 5]', '[3, 4, 5]','[4, 5, 6]','[4.5, 4, 6]', '[5, 5.5, 6]']
    )
)
fig.update_yaxes(title_text="Differenza valore del portafoglio")
fig.update_xaxes(title_text="Maturità degli strumenti di hedging ")  
fig.show()

 #valore portafoglio


Anche in questo caso, vediamo come alcune maturità performano meglio rispetto ad altre. Notiamo molte somiglianze tra le performance anche tra diversi scenari, però è evidente come alcune scelte siano decisamente migliori di altre nel contenere le perdite.

### Value at risk
Per concludere, aumentiamo il numero di scenari di scift simulando N=1000 perturbazioni, variando i parametri del modello tramite il campionamento da una distribuzione normale di media 0 e deviazione standard 0.05.  
Per ogni tripletta di maturità, calcoliamo le differenze dei valori del portafoglio coperto, quindi scegliamo quale 'tripletta di maturità' usare per gli strumenti di hedging, tramite il value-at-risk a livello $ \alpha=0.05$. \
Osserviamo che la coda da penalizzare è quella di sinistra, in quanto differenze di valore negative rappresentano delle perdite. 

In [40]:
numeri_scenari=1000
differenza= np.zeros((len(maturità), numeri_scenari))
differenzarelativa= np.zeros((len(maturità), numeri_scenari))
var=np.zeros(len(maturità))
fig = go.Figure()


for j in range(numeri_scenari):
  r_shift=dict()
  b0_shift= b0+ np.random.normal(0, 0.05)
  b1_shift= b1+  np.random.normal(0, 0.05)
  b2_shift=b2+  np.random.normal(0, 0.05)

  for t in times:
    r_shift[t]= b0_shift + b1_shift*((1- np.exp(-t/tau ))/(t/tau)) + b2_shift*((1-np.exp(- t/tau))/(t/tau) - np.exp(- t/tau) )


  fig.add_trace(go.Scatter(x=list(r_shift.keys()), y=list(r_shift.values()),
                      mode='lines+markers',
                      name='r-shift'+ str(j+1)))  
  
  P_portafoglio_shift, _ =compute_value_portfolio(r_shift, N, F1,T1,F2,T2,F3,T3,F4,T4) 
  
  for i, HT in enumerate(maturità):
    differenza[i,j], differenzarelativa[i,j] = compute_phi_nelson(F,HT,r, r_shift, P_portafoglio, P_portafoglio_shift) 
alpha=0.05
for i in range(len(maturità)):
  var[i] = np.quantile(differenza[i,:], alpha)

optimal_index = np.nanargmax(var)
optimal_decision = maturità[optimal_index]
optimal_var = var[optimal_index]

loss_optimal_var = differenza[optimal_index,:]
fig = go.Figure()
fig.add_trace(go.Histogram(x= loss_optimal_var, histnorm='probability',  marker_color='green',
      opacity=0.90))
fig.add_trace(go.Scatter(
      x=[optimal_var],
      y=[-0.003],
      text=["</b>$V@R_{" + str(alpha) + "}$</b>"],
      mode="text", textfont=dict(
          size=18,
          color="crimson")
      ))

fig.add_shape(type="line",
      x0=optimal_var, y0=0, x1=optimal_var, y1=0.015,
      line=dict(
          color="red",
          width=3,
          dash="dashdot",
      )
  )
fig.update_shapes(dict(xref='x', yref='y'))

fig.update_layout( showlegend=False,
      title = 'La decisione ottimale delle maturità è: '+ '{:.1f},'.format(optimal_decision[0]) +
       '{:.1f},'.format(optimal_decision[1])+ '{:.1f}'.format(optimal_decision[2]),
      autosize=False,
      width=800,
      height=800,
      margin=dict(
          l=50,
          r=50,
          b=100,
          t=100,
          pad=4
      ),  yaxis_title='Count', xaxis_title= 'Differenza del valore del portafoglio coperto'
  )
    
fig.show()

Per concludere, basandoci sul calcolo del Value at risk, la tripletta di maturità degli strumenti di hedging che ci permette di immunizzare meglio il portafoglio ed evitare perdite dovute alla realizzazione dei fattori di rischio è (1,5,6).